In [1]:
import pandas as pd
import numpy as np
import pickle as cp
from sklearn.preprocessing import OneHotEncoder

from keras.models import Sequential
from keras.layers import Dense, Conv1D, Conv2D, LSTM, CuDNNLSTM, Flatten, Dropout, Input, TimeDistributed, Reshape
from keras import optimizers
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
def load_dataset(filename):

    f = open(filename, 'rb')
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print("shapes: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)
    
    print('Dataset loaded successfully')

    return X_train, y_train, X_test, y_test

In [3]:
def slidingWindow(sequence, labels, winSize, step, noNull):

    # Verify the inputs
    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")
    if winSize > len(sequence):
        raise Exception("**ERROR** winSize must not be larger than sequence length.")
 
    # number of chunks
    numOfChunks = ((len(sequence)-winSize)//step)+1
 
    # Do the work
    for i in range(0,numOfChunks*step,step):
        segment = sequence[i:i+winSize]
        seg_labels = labels[i:i+winSize]
        if noNull:
            if seg_labels[-1] != 0:
                yield segment, seg_labels
        else:
            yield segment, seg_labels

In [4]:
def segment_data(X_train, y_train, X_test, y_test, winSize, step, noNull=False):
    assert len(X_train) == len(y_train)
    assert len(X_test) == len(y_test)
    # obtain chunks of data
    train_chunks = slidingWindow(X_train, y_train , winSize, step, noNull)
    test_chunks = slidingWindow(X_test, y_test, winSize, step, noNull)
    
    # segment the data
    train_segments = []
    train_labels = []
    for chunk in train_chunks:
        data = chunk[0]
        labels = chunk[1]
        train_segments.append(data)
        train_labels.append(labels[-1])
    
    test_segments = []
    test_labels = []
    for chunk in test_chunks:
        data = chunk[0]
        labels = chunk[1]
        test_segments.append(data)
        test_labels.append(labels[-1])
        
    return np.array(train_segments), np.array(train_labels), np.array(test_segments), np.array(test_labels)

In [5]:
num_sensors = 113 # number of sensor channels
num_classes = 18 # number of classes 
window_size = 24 # window size
step_size = 12 # half of the sliding window

X_train, y_train, X_test, y_test = load_dataset('oppChallenge_gestures.data')

train_segments, train_labels, test_segments, test_labels = segment_data(X_train, y_train, X_test, y_test,
                                                                        window_size, step_size)
encoder = OneHotEncoder()
train_labels = encoder.fit_transform(train_labels.reshape(-1,1)).toarray()
test_labels = encoder.transform(test_labels.reshape(-1,1)).toarray()

shapes: train (557963, 113), test (118750, 113)
Dataset loaded successfully


In [6]:
train_segments = train_segments.reshape((-1,window_size, num_sensors,1))
test_segments = test_segments.reshape((-1,window_size,num_sensors,1))

In [7]:
model = Sequential()
size_of_kernel = (5,1)
kernel_strides = 1
num_filters = 64
num_lstm_cells = 128
dropout_prob = 0.5
inputshape = (window_size, num_sensors, 1)

In [8]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 activation='relu', input_shape=inputshape, name='1_conv_layer'))

In [9]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 activation='relu', name='2_conv_layer'))

In [10]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 activation='relu', name='3_conv_layer'))

In [11]:
model.add(Conv2D(num_filters, kernel_size=size_of_kernel, strides=kernel_strides,
                 activation='relu', name='4_conv_layer'))

In [12]:
model.add(Reshape((8, num_filters*num_sensors)))

In [13]:
model.add(CuDNNLSTM(num_lstm_cells, return_sequences=True, name='1_lstm_layer'))

model.add(Dropout(dropout_prob, name='2_dropout_layer'))

model.add(CuDNNLSTM(num_lstm_cells, return_sequences=False, name='2_lstm_layer'))

model.add(Dropout(dropout_prob, name='3_dropout_layer'))

model.add(Dense(num_classes, activation='softmax', name='softmax_layer'))

rms = optimizers.RMSprop(lr=0.001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=rms, metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [14]:
for layer in model.layers:
    print(str(layer.name) + ': input shape: ' + str(layer.input_shape) + ' output shape: ' + str(layer.output_shape))

1_conv_layer: input shape: (None, 24, 113, 1) output shape: (None, 20, 113, 64)
2_conv_layer: input shape: (None, 20, 113, 64) output shape: (None, 16, 113, 64)
3_conv_layer: input shape: (None, 16, 113, 64) output shape: (None, 12, 113, 64)
4_conv_layer: input shape: (None, 12, 113, 64) output shape: (None, 8, 113, 64)
reshape_1: input shape: (None, 8, 113, 64) output shape: (None, 8, 7232)
1_lstm_layer: input shape: (None, 8, 7232) output shape: (None, 8, 128)
2_dropout_layer: input shape: (None, 8, 128) output shape: (None, 8, 128)
2_lstm_layer: input shape: (None, 8, 128) output shape: (None, 128)
3_dropout_layer: input shape: (None, 128) output shape: (None, 128)
softmax_layer: input shape: (None, 128) output shape: (None, 18)


In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1_conv_layer (Conv2D)        (None, 20, 113, 64)       384       
_________________________________________________________________
2_conv_layer (Conv2D)        (None, 16, 113, 64)       20544     
_________________________________________________________________
3_conv_layer (Conv2D)        (None, 12, 113, 64)       20544     
_________________________________________________________________
4_conv_layer (Conv2D)        (None, 8, 113, 64)        20544     
_________________________________________________________________
reshape_1 (Reshape)          (None, 8, 7232)           0         
_________________________________________________________________
1_lstm_layer (CuDNNLSTM)     (None, 8, 128)            3769344   
_________________________________________________________________
2_dropout_layer (Dropout)    (None, 8, 128)            0         
__________

In [16]:
batchSize = 100
train_epoches = 50
model.fit(train_segments,train_labels,validation_data=(test_segments,test_labels),epochs=train_epoches,batch_size=batchSize,verbose=1)

print('Calculating score.. ')
score = model.evaluate(test_segments,test_labels,verbose=1)
print(score)
model.save('taskB2_all_Subjects_CNN_LSTM_model.h5')

Train on 46495 samples, validate on 9894 samples
Epoch 1/50
46495/46495 [==============================] - 61s 1ms/step - loss: 1.0147 - acc: 0.7414 - val_loss: 0.4515 - val_acc: 0.8573
Epoch 2/50
46495/46495 [==============================] - 54s 1ms/step - loss: 0.5355 - acc: 0.8201 - val_loss: 0.4024 - val_acc: 0.8721
Epoch 3/50
46495/46495 [==============================] - 54s 1ms/step - loss: 0.4442 - acc: 0.8466 - val_loss: 0.3622 - val_acc: 0.8876
Epoch 4/50
46495/46495 [==============================] - 55s 1ms/step - loss: 0.3911 - acc: 0.8632 - val_loss: 0.3303 - val_acc: 0.8890
Epoch 5/50
46495/46495 [==============================] - 55s 1ms/step - loss: 0.3551 - acc: 0.8754 - val_loss: 0.3679 - val_acc: 0.8892
Epoch 6/50
46495/46495 [==============================] - 55s 1ms/step - loss: 0.3242 - acc: 0.8881 - val_loss: 0.3372 - val_acc: 0.8947
Epoch 7/50
46495/46495 [==============================] - 54s 1ms/step - loss: 0.2930 - acc: 0.8992 - val_loss: 0.3612 - val_acc:

NameError: name 'reshaped_test' is not defined

In [17]:
predictions = model.predict(test_segments)

In [21]:
# F1-score measure
from sklearn.metrics import f1_score
num_classes = 18
class_predictions = []
class_true = []
tot_labels = 0.0
count = 0.0
for pair in zip(predictions, test_labels):
    class_predictions.append(np.argmax(pair[0]))
    class_true.append(np.argmax(pair[1]))
    if np.argmax(pair[0]) == np.argmax(pair[1]):
        count += 1.0
    tot_labels += 1.0
    
print('Standard accuracy is ' + str(count/tot_labels))    

unique, counts = np.unique(class_true, return_counts=True)
counted_labels = dict(zip(unique, counts))
f1_scores = f1_score(class_predictions, class_true, average=None)

tot_f1_score = 0.0
weights_sum = 0.0
for i in range(num_classes):
    labels_class_i = counted_labels[i]
    weight_i = labels_class_i / tot_labels
    weights_sum += weight_i
    tot_f1_score += f1_scores[i]*weight_i
    
print('The computed f1-score is {}'.format(tot_f1_score))
print('The f1-score with sklearn function is {}'.format(f1_score(class_true, class_predictions, average='weighted')))

Standard accuracy is 0.8927632908833637
The computed f1-score is 0.8945253356596347
The f1-score with sklearn function is 0.8945253356596349


In [ ]:
pred_df = pd.DataFrame(predictions)
pred_df.to_csv('preds_test.csv', header=False, index=False)

In [ ]:
true_df = pd.DataFrame(testY)
true_df.to_csv('true_test.csv', header=False, index=False)